In [5]:
import numpy as np
import pandas as pd
import pyfpgrowth
from pyhive import hive

In [4]:
# !pip install pyfpgrowth
# !pip install thrift

In [9]:
# !pip install pyhive
!pip install sasl
!pip install thrift_sasl

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Running setup.py install for sasl ... error
    Complete output from command /opt/conda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-aprk82dh/sasl/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-e1wlnnf2-record/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    cre

In [6]:
def ResultIter(cursor, arraysize=1000):
    'An iterator that uses fetchmany to keep memory usage down'
    while True:
        results = cursor.fetchmany(arraysize)
        if not results:
            break
        for result in results:
            yield result

In [8]:
#conn = hive.Connection(host="mdc2pr300", port=10000, auth="KERBEROS",database="mcom",kerberos_service_name="hive")
conn = hive.Connection(host="mdc2pr191", port=10000)

ModuleNotFoundError: No module named 'sasl'

In [139]:
cursor = conn.cursor()

In [126]:
# cursor.execute("""
# select
# trn.tran_id
# ,trn.appl_xref
# ,trn.intr_xref
# ,trn.item_upc as upc
# ,upc.keyf_products as webId
# from t_idm_ddw_trnsctn_itm as trn
# join t_idm_mdc_upc as upc on
#     upc.upc = trn.item_upc
# where dt >= 20190201
# and fill_meth != 'RETURN'
# and appl_id = 'MCOM'
# and itm_price > 1
# limit 100
#  """)


In [140]:
cursor.execute("""
select
trn.tran_id
,trn.item_upc as upc
from t_idm_ddw_trnsctn_itm as trn
where dt = 20190307
and fill_meth != 'RETURN'
and appl_id = 'MCOM'
and itm_price > 1
""")

In [141]:
df = pd.DataFrame(ResultIter(cursor))

In [142]:
# df.columns = ['tranId','applXref','intrXref','upc','webId']
df.columns = ['tranId','upc']

In [144]:
df.describe

<bound method NDFrame.describe of                   tranId            upc
0       1000000720607340   726113766441
1       1000000720612204  5907755376075
2       1000000720612878   191045301679
3       1000000720613234   732995159301
4       1000000720613574   192399473760
5       1000000720613869   190618458178
6       1000000720614009   805004083277
7       1000000720614284   191454527530
8       1000000720614307   715676277471
9       1000000720614664   726895027556
10      1000000720614876    85447989392
11      1000000720614995   732994452793
12      1000000720615016   191232287847
13      1000000720615039  7613402342338
14      1000000720615314   706254746786
15      1000000720615337   191013986082
16      1000000720615592   706258011125
17      1000000720615615   715676637435
18      1000000720615694   716357851799
19      1000000720615717   716357985388
20      1000000720615843   716357883073
21      1000000720615845    27131987840
22      1000000720615931   716170102764
23    

In [145]:
a = df.groupby('tranId')['upc'].apply(list)

In [146]:
transactions = a.tolist()

In [147]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 5)

In [148]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)

In [150]:
len(rules)

2156

In [105]:
import csv

In [114]:
with open('rules.csv','w') as results:
    wr = csv.writer(results, dialect='excel')
    for key, val in rules.items():
        wr.writerow([key,val])

In [109]:
rules[0]

KeyError: 0

In [113]:
import csv
 
dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}
w = csv.writer(open("output.csv", "w"))
for key, val in dict.items():
    w.writerow([key, val])

In [164]:
cursor.execute("""
select
order_id
,product_id
from mcom.t_ddf_cart_item_purchase 
where dt = 20190307
and base_price >3
""")

In [165]:
df1 = pd.DataFrame(ResultIter(cursor))


In [166]:
df1.columns = ['orderId','prodId']

In [167]:
a = df1.groupby('orderId')['prodId'].apply(list)

In [168]:
transactions1 = a.tolist()

In [169]:
patterns = pyfpgrowth.find_frequent_patterns(transactions1, 5)

In [170]:
rules1 = pyfpgrowth.generate_association_rules(patterns, 0.7)

In [171]:
len(rules1)

4787

In [172]:
rules1

{('5728855',): (('5728866',), 1.0),
 ('6768036',): (('6460533',), 1.0),
 ('737883',): (('780118',), 1.0),
 ('7908796',): (('7908797',), 1.0),
 ('7908797',): (('7908796',), 0.7142857142857143),
 ('2915942',): (('7910406',), 1.2),
 ('6700554',): (('7021752',), 1.6),
 ('7021752',): ((), 0.75),
 ('7691237',): (('7691245',), 1.0),
 ('7691245',): (('7691237',), 0.8333333333333334),
 ('506610',): (('2112827',), 1.0),
 ('1926010', '7063897'): (('5606665', '7063908'), 1.2),
 ('1926010', '7063908'): (('5606665',), 1.0),
 ('1926010', '5606665', '7063897'): (('7063908',), 1.2),
 ('1926010', '5606665', '7063908'): (('7063897',), 1.0),
 ('1926010', '7063897', '7063908'): (('5606665',), 1.0),
 ('6937574',): (('6937578',), 1.2),
 ('6937578',): (('6937574',), 0.75),
 ('4293174',): (('2837067',), 1.0),
 ('6332948',): (('6332947',), 1.0),
 ('7989029',): (('8022627',), 1.0),
 ('8022627',): (('7989029',), 0.8333333333333334),
 ('6279434',): (('4921299',), 1.0),
 ('7283483',): (('8040093',), 1.0),
 ('196576